# Знакомство с Pymol

Для выполнения практикума крайне желательно установить jupyter notebook и pymol. Команды для установки через conda:

```
conda create -n schrodinger
conda activate schrodinger
# Это команды для создания отдельного environment для нужных модулей
# Так будет меньше вероятность их конфликта с уже установленными.

```

```
conda install -c schrodinger pymol
conda install -c anaconda jupyter ipykernel
#ipython kernel install --user
```

Данный ноутбук следует скачать и запускать локально через jupyter notebook или vscode

Также можно запускать его прямо в colab, но интерактивности уже не будет

### 1. Запуск Pymol

####1.1 Запуск pymol из ноутбука

In [ ]:
import __main__
__main__.pymol_argv = [ 'pymol', '-x' ]

import pymol
pymol.finish_launching() # для локального запуска
from pymol import cmd,stored

#### 1.2. Запуск в режиме rpc сервера

Если пункт 1.1 не получился, можно использовать pymol в режиме rpc сервера. Для этого надо запустить pymol в терминале следующий командой:

```
pymol -x -R
```

А затем подключиться к нему из ноутбука

In [ ]:
from xmlrpc.client import ServerProxy

cmd = ServerProxy(uri="http://localhost:9123/RPC2")

В обоих случаях должно открыться окно pymol, в которое можно подавать команды через cmd.

#### 1.3. Запуск ноутбука из колаба

Если ничто из вышеперечисленного не вышло, можно запускать ноутбук в google colab

In [ ]:
!sudo apt-get install pymol

In [ ]:
import pymol
from pymol import cmd,stored

### 2. Визуализация

In [ ]:
from IPython.display import Image
import time

cmd  передает командной строке pymol различные команды

In [ ]:
cmd.reinitialize()

In [ ]:
cmd.fetch('1lmp') # загрузка структуры из pdb

cmd.png('prak1_1.png') # сохранение сессии pymol в виде картинки
time.sleep(2)
Image(filename='prak1_1.png') # вывод картинки в ноутбук


In [ ]:
cmd.do('bg wheat') # поменять цвет фона

In [ ]:
cmd.hide('all')
cmd.show('lines','all') # показать все в стиле sticks

#cmd.png('prak1_1.png')
#Image(filename='prak1_1.png') # команды на случай запуска в колабе, чтобы видеть происходящее

In [ ]:
import time

for i in range(1,11):
    cmd.orient(f'resi {i}')
    # или
    # cmd.zoom(f'resi {i}')
    time.sleep(.5)
    # поочередно наводить камеру на каждый из первых 10 остатков

#cmd.png('prak1_1.png')
#Image(filename='prak1_1.png')

Команды можно подавать в cmd и одной строкой

In [ ]:
cmd.do('''
bg black
hide all
show lines, all
set line_width, 3
color violet, name N*
orient all
''')
#cmd.png('prak1_1.png')
#Image(filename='prak1_1.png')

Задание: покажите цепь А (белковую) в стиле cartoon, цепь B (олигосахарид) - в стиле sticks, а молекулы растворителя (HOH) - в виде сфер радиуса 0.2. Наведите камеру на цепь B. Покажите результат в ноутбуке.

In [ ]:
cmd.do('''
       
...
       
''')
cmd.png('prak1_1.png')
Image(filename='prak1_1.png')

### 2. Итерация по остаткам

Стало ясно, как передать команды из кода в pymol. Что делать, если нужно получить информацию из pymol? Например, получить имена всех остатков в структуре.

In [ ]:
stored.r = [] # создаем переменную, к которой у pymol будет доступ

cmd.iterate('chain A and name CA','stored.r.append(resn)')
# метод iterate получает на вход набор атомов
# (в данном случае - все CA атомы цепи А),
# и к каждому атому применяет команду
# (в данном случае - добавить название остатка в созданный ранее список)

In [ ]:
' '.join(stored.r)

In [ ]:
# примечание. запись типа:
# f'abcd{l}' - это метод форматирования строки,
# когда в середину строки надо вставить значение переменной.
# перед строкой пишется f,
# а сама переменная пишется в нужном месте строки в фигурных скобках.
# поскольку командная строка пимола принимает команды в виде строк,
# данный метод тут очень полезен и удобен.
# подробнее https://pythonist.ru/formatirovanie-strok-v-python-format-i-f-stroki/

f'variable l is {l}'

Задание: каждому остатку в цепи A присвоить цвет, изменяющийся по градиенту от синего до красного, отобразить структуру в виде cartoon


In [ ]:

l=len(stored.r)
for i in range(l):
    ...
    #cmd.set_color(<color name>, [r,g,b]) # команда, задающая определенному цвету свое имя, по которому к нему потом можно обращаться
    ...

cmd.png('prak1_1.png')
Image(filename='prak1_1.png')

### 3. Создание видео

Создадим видео, в котором камера поочередно наводится на каждый остаток

In [ ]:
cmd.center('all')

cmd.mset(f'1 x {l*2}') # создаем нужное кол-во кадров (по 2 на каждый остаток)

for i in range(l):

    cmd.frame(2*(i+1)) # определяем номер кадра
    cmd.orient(f'chain A and resi {i}+{i+7}') # наводим камеру сразу на 7 остатков, а то слишком близко выходит
    cmd.mview('store')  # сохраняем кадр

cmd.mview('reinterpolate') # сглаживаем движения камеры

Можно сохранить кадры видео в виде отдельных png картинок

In [ ]:
cmd.mpng('movie_1_', 0, l*2)

И конвертировать в видео, например, программой ffmpeg (есть альтернативы в библиотеках PIL, сv2 и пр.)

In [ ]:
import os, sys
os.system("ffmpeg -r 1 -i movie_1_*.png -vcodec mpeg4 -y movie_1.mp4")

Вывести видео в ноутбук

In [ ]:
from IPython.display import HTML

HTML("""
<video width="320" height="240" controls>
  <source src="movie_1.mp4" type="video/mp4">
</video>
""")

# обычно плеер никогда не работает, у кого заработает, тот молодец
# попробуйте использовать markdown ячейки

In [ ]:
# как вариант, сделать гифку
cmd.movie.produce('movie_1.gif')

In [ ]:
Image('movie_1.gif')

Задание: найти в pdb (https://www.rcsb.org/) еще одну структуру лизоцима и снять видео с наложением одной структуры на другую (команда alignto)

### 4. Редактирование

In [ ]:
cmd.center('resi 129')
cmd.hide('all')
cmd.show('sticks', 'all')

#cmd.png('prak1_1.png')
#Image(filename='prak1_1.png')

Pymol предоставляет массу инструментов для редактирования структуры. Большинство из них доступно через GUI, но есть и те, что можно запускать через командную строку

In [ ]:
cmd.edit('resi 129 and name C') # выделить атом для редактирования

In [ ]:
cmd.editor.attach_amino_acid('pk1','tyr',1,1) # приделать к нему тирозин

In [ ]:
cmd.edit('resi 129 and name C', 'resi 130 and name N') # выделить пару атомов для редактирования ковалентной связи
cmd.torsion(180) # повернуть вокруг связи на 180 градусов

Задание: постройте полиаланиновую альфа-спираль длиной 50 аминокислот. Учтите, что средние значения углов для альфа-спирали: phi=-64,23 psi=-39,08

In [ ]:
cmd.delete('all')
cmd.fragment('ala') # создать одинокий аланин
...